# Deploying functions 

In [185]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/

In [186]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml --annotation topic="aeneas" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=15 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [187]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=5 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud



In [188]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f getobject.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject



In [189]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f pubrabbitmq.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [190]:
!ssh ubuntu@172.17.141.197 "faas describe aeneas"

Name:               aeneas
Status:             Ready
Replicas:           1
Available Replicas: 1
Invocations:        40510
Image:              shivupoojar/pythonaeneasrabbitmq:latest
Function Process:   python index.py
URL:                http://127.0.0.1:8080/function/aeneas
Async URL:          http://127.0.0.1:8080/async-function/aeneas
Labels:
 com.openfaas.scale.min: 1
 com.openfaas.scale.target-proportion: 0.90
 com.openfaas.scale.type: rps
 faas_function: aeneas
 uid: 266714417
 com.openfaas.scale.max: 15
Annotations:
 prometheus.io.scrape: false
 topic: aeneas
Requests:  CPU: 1
 Memory: 512Mi


# Deploying Rabbitmq Connector 

In [191]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [192]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl delete  -f configmap.yaml"

deployment.apps "rabbitmq-connector" deleted


In [193]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply  -f configmap.yaml"

deployment.apps/rabbitmq-connector created


In [194]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS              RESTARTS         AGE
nats-5b77986b8-48p2x                  1/1     Running             4 (8d ago)       44d
queue-worker-7d8668444f-tmq7b         1/1     Running             5 (8d ago)       30d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running             4 (8d ago)       30d
basic-auth-plugin-78b958969c-tnpck    1/1     Running             4 (8d ago)       30d
gateway-bf4f5d6f7-4q7kn               2/2     Running             10 (6d23h ago)   30d
prometheus-7678c644d6-k28wr           1/1     Running             0                2d7h
rabbitmq-connector-64cbd548f5-gdsqk   0/1     ContainerCreating   0                2s


# Experiments 

In [195]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [196]:
scenario = "rps_keda"
user=10
iteration=0.08
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=300


In [197]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [198]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    client.remove_object(bucket_name, obj.object_name)

In [199]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 300s  --users 10 --spawn-rate 0.08

In [200]:
time.sleep(80)

In [201]:
df= pd.read_csv('input_data.csv')
print(df)

                           intime
0    '2023-01-19 18:10:46.784608'
1    '2023-01-19 18:10:57.361472'
2    '2023-01-19 18:10:59.275737'
3    '2023-01-19 18:11:04.348511'
4    '2023-01-19 18:11:08.825738'
..                            ...
236  '2023-01-19 18:15:40.446405'
237  '2023-01-19 18:15:40.480791'
238  '2023-01-19 18:15:40.662053'
239  '2023-01-19 18:15:43.329973'
240  '2023-01-19 18:15:43.907219'

[241 rows x 1 columns]


In [202]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)

In [210]:
outtime2 = df['intime'].shape[0]


In [213]:
df['outtime']= outtime[:-outtime2]

df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-19 18:10:46.784608,2023-01-19 18:11:02.813
1,2023-01-19 18:10:57.361472,2023-01-19 18:11:05.358
2,2023-01-19 18:10:59.275737,2023-01-19 18:11:09.641
3,2023-01-19 18:11:04.348511,2023-01-19 18:11:13.858
4,2023-01-19 18:11:08.825738,2023-01-19 18:11:18.083
...,...,...
236,2023-01-19 18:15:40.446405,2023-01-19 18:15:43.624
237,2023-01-19 18:15:40.480791,2023-01-19 18:15:44.064
238,2023-01-19 18:15:40.662053,2023-01-19 18:15:45.726
239,2023-01-19 18:15:43.329973,2023-01-19 18:15:47.776


0 days 00:05:01.098392


In [224]:
outtime=df['outtime']

In [227]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))

In [228]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))

df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-19 18:10:46.784608,2023-01-19 18:11:02.813,- 0.,0.0437,0.0330
1,2023-01-19 18:10:57.361472,2023-01-19 18:11:05.358,5.34,0.0409,0.0362
2,2023-01-19 18:10:59.275737,2023-01-19 18:11:09.641,5.98,0.0490,0.0468
3,2023-01-19 18:11:04.348511,2023-01-19 18:11:13.858,5.20,0.0605,0.0324
4,2023-01-19 18:11:08.825738,2023-01-19 18:11:18.083,4.93,0.0367,0.0500
...,...,...,...,...,...
236,2023-01-19 18:15:40.446405,2023-01-19 18:15:43.624,5.17,0.0422,0.0514
237,2023-01-19 18:15:40.480791,2023-01-19 18:15:44.064,4.37,0.0358,0.0443
238,2023-01-19 18:15:40.662053,2023-01-19 18:15:45.726,7.32,0.0396,0.0332
239,2023-01-19 18:15:43.329973,2023-01-19 18:15:47.776,7.29,0.0458,0.0340


In [229]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
df.to_csv(str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')

ValueError: could not convert string to float: '- 0.'

In [ ]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

In [ ]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [ ]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

In [ ]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [ ]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [ ]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [ ]:
promo_metrics

In [ ]:
df_metrics = pd.read_csv("overall_rps.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)

In [ ]:
df_metrics.to_csv("overall_rps.csv",index=False)
display(df_metrics)

In [ ]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
 #   data=res.get('data').get('result')[0].get('value')[1]
    return res

In [ ]:

# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)


stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)
